In [2]:
!pip install recipe-scrapers

  Obtaining dependency information for recipe-scrapers from https://files.pythonhosted.org/packages/b9/5e/5873ad103c4131705fa145a6e0bedb188a026823ec0f7bb4ad09b77b173a/recipe_scrapers-14.42.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 82.5 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for extruct>=0.8.0 from https://files.pythonhosted.org/packages/ee/ba/6cda7321aa72f7438d8e98325296d1cfa9da3b0f3e1416612389ffc84cc9/extruct-0.16.0-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 97.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 43.8 kB/s eta 0:00:00a 0:00:02
  Obtaining dependency information for mf2py from https://files.pythonhosted.org/packages/86/e9/feebadcc2ffd04f13b4d261c07ce0a89b562e66984946f5bac6f2cf46703/mf2py-1.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for w3lib from https://files.pythonhosted.org/packages/82/e2/dcf8573d71

In [9]:
from recipe_scrapers import scrape_me
import requests
from bs4 import BeautifulSoup
import pandas as pd
import concurrent.futures
from tqdm import notebook, tqdm
import time
from requests.exceptions import ConnectionError
import random

In [37]:
# categories = [
#     "https://www.allrecipes.com/recipes/",
#     "https://www.allrecipes.com/recipes/17562/dinner/",
#     "https://www.allrecipes.com/recipes/1116/fruits-and-vegetables/",
#     "https://www.allrecipes.com/recipes/156/bread/",
#     "https://www.allrecipes.com/recipes/1642/everyday-cooking/",
#     "https://www.allrecipes.com/recipes/17561/lunch/",
#     "https://www.allrecipes.com/recipes/17567/ingredients/",
#     "https://www.allrecipes.com/recipes/236/us-recipes/",
#     "https://www.allrecipes.com/recipes/76/appetizers-and-snacks/",
#     "https://www.allrecipes.com/recipes/77/drinks/",
#     "https://www.allrecipes.com/recipes/78/breakfast-and-brunch/",
#     "https://www.allrecipes.com/recipes/79/desserts/",
#     "https://www.allrecipes.com/recipes/80/main-dish/",
#     "https://www.allrecipes.com/recipes/81/side-dish/",
#     "https://www.allrecipes.com/recipes/82/trusted-brands-recipes-and-tips/",
#     "https://www.allrecipes.com/recipes/84/healthy-recipes/",
#     "https://www.allrecipes.com/recipes/85/holidays-and-events/",
#     "https://www.allrecipes.com/recipes/86/world-cuisine/",
#     "https://www.allrecipes.com/recipes/88/bbq-grilling/",
#     "https://www.allrecipes.com/recipes/92/meat-and-poultry/",
#     "https://www.allrecipes.com/recipes/93/seafood/",
#     "https://www.allrecipes.com/recipes/94/soups-stews-and-chili/",
#     "https://www.allrecipes.com/recipes/95/pasta-and-noodles/",
#     "https://www.allrecipes.com/recipes/96/salad/"
# ]


categories = [
    "https://www.allrecipes.com/recipes/17485/everyday-cooking/quick-and-easy/breakfast-and-brunch/?page=34",
    "https://www.allrecipes.com/recipe/20144/banana-banana-bread/"
]

In [38]:
def scrape_recipe_links(URL):
    i = 0
    recipes = []
    category = URL.strip('/').split('/')[len(URL.strip('/').split('/')) - 1]

    
    while(True):
        try:
            currURL = URL + '?page=' + str(i)
            HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
            print(currURL)
            page = requests.get(currURL, headers=HEADERS)#, proxies={'http': 'http://50.192.195.69:52018', 'https': 'https://154.72.202.62:53281'})

            i = i + 1
            
            soup = BeautifulSoup(page.content, 'html.parser')
            div = soup.find(id='fixedGridSection')
            articles = div.findChildren("article" , class_="fixed-recipe-card",recursive=False)
            links = []
            for article in articles:
                div = article.find_all(class_='grid-card-image-container')
                a = div[0].find("a", recursive=False)
                link = a['href']
                recipes.append(link)
                with open('recipe_links.txt', 'a') as f:
                  f.write(link + ',' + category + ';')
                  f.close()
            time.sleep(random.randrange(1,3))

        except KeyboardInterrupt:
            raise
        except ConnectionError as e:
            print(e)
            time.sleep(random.randrange(1,3))
            continue
        except:
            # raise
            break
        
    return recipes, category

# for category in [categories[0]]:
#     r = scrape_recipe_links(category)

# recipe_links=[]

f = open('recipe_links.txt', 'w+')
f.close()

threads = min(25, len(categories))
print(threads)

with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    r = list(tqdm(executor.map(scrape_recipe_links, categories), total=len(categories)))

print(r)



2
https://www.allrecipes.com/recipes/17485/everyday-cooking/quick-and-easy/breakfast-and-brunch/?page=34?page=0
https://www.allrecipes.com/recipe/20144/banana-banana-bread/?page=0


100%|██████████| 2/2 [00:01<00:00,  1.16it/s]

[([], '?page=34'), ([], 'banana-banana-bread')]


In [ ]:
pd.DataFrame(r).to_csv('/content/drive/My Drive/Data Science Projects/RecipeApp/data/myData/ALL_RECIPES.csv')

NameError: name 'pd' is not defined